In [ ]:
%matplotlib inline
import numpy as np
import scipy
import scipy.stats
import operator
import pandas as pd
from __future__ import print_function
import os
import cPickle as pickle
import math
from time import time
from datetime import datetime
import matplotlib.pyplot as plt
np.random.seed(1337)  # for reproducibility

nb_epoch = 500  # number of epoch at training stage
nb_epoch_cont = 500  # number of epoch at training (cont) stage
batch_size = 15  # batch size

lr = 0.001  # learning rate
len_closeness = 15  # length of closeness dependent sequence
nb_residual_unit = 4   # number of residual units
year_test = 2006
nb_flow = 1  # there are two types of flows: new-flow and end-flow
path_result = 'RET'
path_model = 'MODEL'

data_path = '/notebooks/workspace/flood/www.ncei.noaa.gov/data/precipitation-persiann/'

In [ ]:
from persiann import sample_construction
if os.path.isfile('X.npy') is False: 
    print ('start creating samples.')
    sc = sample_construction(data_path=data_path,end_year=2016)
    sc.create_samples(target_river = 'G')

In [ ]:
E = np.load('E.npy') #external input
X = np.load('X.npy')
Y = pd.read_pickle('Y_df.pkl')
map_height, map_width = X.shape[2], X.shape[3]
print('E: ',E.shape,'X: ',X.shape,'Y: ',Y.shape)

In [ ]:
def distribution_plot(y):
    overall_min = np.min(y)
    overall_max = np.max(y)
    #print (overall_max,overall_min)
    bins = np.linspace(overall_min, overall_max, 50)
    #print(bins)   
    fig = plt.figure()
    fig.set_size_inches(15, 10)
    print (y.shape)
    n, bins, patches = plt.hist(y, bins, alpha=0.5, density=True, facecolor='g')#log=True
    #print (n)
    #print (bins)
    #print(patches)
#(x, 50, density=True, facecolor='g', alpha=0.75)
    #print(bins)
    #dist_names = ['rayleigh', 'norm', 'pareto']
    #dist_names = ['alpha', 'anglit', 'arcsine', 'beta', 'betaprime', 'bradford', 'burr', 'cauchy', 'chi', 'chi2', 'cosine', 'dgamma', 'dweibull', 'erlang', 'expon', 'exponweib', 'exponpow', 'f', 'fatiguelife', 'fisk', 'foldcauchy', 'foldnorm', 'frechet_r', 'frechet_l', 'genlogistic', 'genpareto', 'genexpon', 'genextreme', 'gausshyper', 'gamma', 'gengamma', 'genhalflogistic', 'gilbrat', 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy', 'halflogistic', 'halfnorm', 'hypsecant', 'invgamma', 'invgauss', 'invweibull', 'johnsonsb', 'johnsonsu', 'ksone', 'kstwobign', 'laplace', 'logistic', 'loggamma', 'loglaplace', 'lognorm', 'lomax', 'maxwell', 'mielke', 'nakagami', 'ncx2', 'ncf', 'nct', 'norm', 'pareto', 'pearson3', 'powerlaw', 'powerlognorm', 'powernorm', 'rdist', 'reciprocal', 'rayleigh', 'rice', 'recipinvgauss', 'semicircular', 't', 'triang', 'truncexpon', 'truncnorm', 'tukeylambda', 'uniform', 'vonmises', 'wald', 'weibull_min', 'weibull_max', 'wrapcauchy']
    #dist_names = ['invgauss', 'foldcauchy', 'lomax', 'halfcauchy', 'pareto', 'alpha', 'genexpon', 'expon', 'gompertz', 'halflogistic', 'genhalflogistic', 'halfnorm', 'foldnorm', 'gilbrat', 'uniform', 'ksone', 'truncnorm', 'wald', 'bradford', 'triang', 'truncexpon', 'semicircular', 'kstwobign', 'rice', 'rayleigh', 'frechet_l', 'wrapcauchy', 'gumbel_r', 'genlogistic', 'weibull_max', 'maxwell', 'anglit', 'norm', 'loggamma', 'logistic', 'hypsecant', 'cauchy', 'gumbel_l', 'powernorm', 'laplace', 'reciprocal', 'cosine']
    dist_names = ['pareto','norm','alpha','logistic', 'hypsecant', 'cauchy', 'gumbel_l', 'powernorm', 'laplace', 'reciprocal']#'reciprocal'
    #dist_names = ['gumbel_l','logistic','norm','alpha','cauchy', 'laplace']# 
    #dist_names = ['pareto','norm','genpareto', 'halfnorm', 'expon' ]
    #dist_names = ['lognorm']
    #dist_names = ['arcsine', 'truncexpon', 'semicircular', 'anglit', 'triang', 'genhalflogistic', 'lomax', 'expon', 'betaprime', 'exponpow', 'loglaplace', 'fisk', 'exponweib', 'invweibull', 'gumbel_r', 'genlogistic',  'genpareto',  'foldnorm', 'kstwobign','halfcauchy',  'rayleigh',  'rice', 'maxwell',  'cosine', 'mielke',  'genexpon',  'dgamma']
    #dist_names = ['truncexpon', 'semicircular', 'anglit', 'triang', 'genhalflogistic', 'loglaplace', 'betaprime','fisk', 'exponweib', 'invweibull', 'gumbel_r', 'genlogistic',  'genpareto',  'foldnorm', 'kstwobign','halfcauchy',  'rayleigh',  'rice', 'maxwell',  'cosine', 'mielke',  'genexpon',  'dgamma']
    
    dic = {}
    print('here111111')
    for dist_name in dist_names:
        dist = getattr(scipy.stats, dist_name)
        param = dist.fit(y)
        print('here2222222')
        pdf_fitted = dist.pdf(bins, *param[:-2], loc=param[-2], scale=param[-1])       
        NLL = -np.sum(np.log(pdf_fitted))
        print ("Dist: {0}, NLL: {1}".format(dist_name, NLL))
        dic[dist_name]=NLL
        plt.plot(bins, pdf_fitted, label=dist_name+', NLL: '+str(NLL))
    sorted_dic = sorted(dic.items(), key=operator.itemgetter(1))
    print (sorted_dic)
    #plt.xlim(xmin,xmax)
    #plt.ylim(ymin,ymax)
    plt.legend(loc='upper right')
    plt.xlabel('Daily Stream Flow')
    plt.ylabel('Number of Days')
    plt.show()
    return sorted_dic
#plt.xticks(bins, ["2^%s" % i for i in bins])

In [ ]:
#ymax,ymin = 450,0
#xmax,xmin = 2.2,0
#xmax,xmin = 2200,0
#distribution_plot(y,omax,omin)
data = Y #np.array(np.log(1+Y))#np.log(0.5+y)
sorted_dic = distribution_plot(data)
print ('min: '+str(np.min(data)))
print ('max: '+str(np.max(data)))
print ('mean: '+str(np.mean(data)))
print ('median: '+str(np.median(data)))
print ('standard deviation: '+str(np.std(data)))
print ('75% percentile: '+str(np.percentile(data,75)))
ratio = (np.max(data)-np.percentile(data,75))/(np.max(data) - np.min(data))
print ('>75% percentile ratio= '+str(ratio))
#data =np.log(1+y)/np.log(1.6) 
#distribution_plot(data,omax,omin)
#data =np.log(1+y)/np.log(1.5) 
#distribution_plot(data,omax,omin)
#data =np.log(1+y)/np.log(1.1) 
#distribution_plot(data,omax,omin)